#**Exercício 2 - Aula 2**

### Dado o dataset de produtos [1], desenvolva os seguintes pipelines:

[1] - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv

Obs.: em todos os pipelines use:
- normalização renovendo valores faltantes
- criem uma nova coluna concatenando as colunas nome e descrição.
- randon_state igual a 42 para permitir a comparação com seus colegas e separe uma amostra de 30% para teste.


In [1]:
# Pacotes utilizados
import pandas as pd
from sklearn.model_selection import train_test_split

# carregar dataframe
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')

# limpeza inicial (normalização)
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']

# divisão da amostra entre treino e teste
df_train, df_test = train_test_split(
      df,
      test_size = 0.3,
      random_state = 42
  )

### 2. ) Treine um modelo de classificação LogisticRegression do pacote scikit-learn para classificar os produtos em suas categorias, com as seguintes configurações:

- 2.1 Contagem de termos simples com unigrama e com stop-words.
- 2.2 Contagem de termos simples com unigrama + brigrama e sem stop-words.
- 2.3 TF-IDF com unigrama e sem stop-words.
- 2.4 TF-IDF com unigrama e sem stop-words em textos lematizados.

Extra:
- 2.5 Contagem de termos simples (BoW) com unigrama, sem stop-words (combinando Spacy e NLTK) em textos com apenas verbos lematizados.

Dica: crie uma função para lematizar o texto usando o Spacy, não esqueça de usar o POS-Tag quando necessário.

In [2]:
# 2.1 Contagem de termos simples com unigrama e com stop-words.
from sklearn.feature_extraction.text import CountVectorizer

# vetorização
vect = CountVectorizer(ngram_range=(1,1))
vect.fit(df_train.texto)
text_vect_train = vect.transform(df_train.texto)

x_train = text_vect_train
y_train = df_train["categoria"]

from sklearn.linear_model import LogisticRegression

# treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
x_test = vect.transform(df_test.texto)
y_test = df_test["categoria"]

model.score(x_test, y_test)

0.9805714285714285

In [3]:
# 2.2 Contagem de termos simples com unigrama + brigrama e sem stop-words.
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

stops = nltk.corpus.stopwords.words('portuguese')

# vetorização
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops)
vect.fit(df_train.texto)
text_vect_train = vect.transform(df_train.texto)

x_train = text_vect_train
y_train = df_train["categoria"]

from sklearn.linear_model import LogisticRegression

# treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
x_test = vect.transform(df_test.texto)
y_test = df_test["categoria"]

model.score(x_test, y_test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0.9851428571428571

In [4]:
# 2.3 TF-IDF com unigrama e sem stop-words.
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')

stops = nltk.corpus.stopwords.words('portuguese')

# vetorização
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
vect.fit(df_train.texto)
text_vect_train = vect.transform(df_train.texto)

x_train = text_vect_train
y_train = df_train["categoria"]

from sklearn.linear_model import LogisticRegression

# treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
x_test = vect.transform(df_test.texto)
y_test = df_test["categoria"]

model.score(x_test, y_test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0.984

In [ ]:
#!pip install spacy
!python -m spacy download pt_core_news_sm

In [6]:
# 2.4 TF-IDF com unigrama e sem stop-words em textos lematizados.
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
import spacy
nlp = spacy.load('pt_core_news_sm')

stops = nltk.corpus.stopwords.words('portuguese')

# função de lematização completa do documento
def lemmatizer_text(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)

# aplica a lematização no dataframe de treino criando um nova coluna
df_train['text_lemma'] = df_train.texto.apply(lemmatizer_text)

# vetorização
vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
vect.fit(df_train.text_lemma)
text_vect_train = vect.transform(df_train.text_lemma)

x_train = text_vect_train
y_train = df_train["categoria"]

from sklearn.linear_model import LogisticRegression

# treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
# aplica a lematização no dataframe de treino criando um nova coluna
df_test['text_lemma'] = df_test.texto.apply(lemmatizer_text)
x_test = vect.transform(df_test.text_lemma)
y_test = df_test["categoria"]

model.score(x_test, y_test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0.9828571428571429

In [7]:
# Extra:
# 2.5 Contagem de termos simples (BoW) com unigrama, sem stop-words (combinando Spacy e NLTK) em textos com apenas verbos lematizados.
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
import spacy
nlp = spacy.load('pt_core_news_sm')

# stopwords do SpaCy e NLTK combinadas
stops = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))

# função de lematização para os verbos do documento
def lemmatizer_verbs(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.text)
  return " ".join(sent)

# aplica a lematização no dataframe de treino criando um nova coluna
df_train['text_lemma_verbs'] = df_train.texto.apply(lemmatizer_verbs)

# vetorização
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_train.text_lemma_verbs)
text_vect_train = vect.transform(df_train.text_lemma_verbs)

x_train = text_vect_train
y_train = df_train["categoria"]

from sklearn.linear_model import LogisticRegression

# treinamento do modelo
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

# teste do modelo
# aplica a lematização no dataframe de treino criando um nova coluna
df_test['text_lemma_verbs'] = df_test.texto.apply(lemmatizer_verbs)
x_test = vect.transform(df_test.text_lemma_verbs)
y_test = df_test["categoria"]

model.score(x_test, y_test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0.9874285714285714